In [1]:
!pip install ipywidgets numpy matplotlib IPython yfinance

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from ipywidgets import VBox, HBox, Text, Button, Output, Layout, Dropdown, Label
import sys
import os
import warnings
from contextlib import contextmanager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.7 MB/s eta 0:00:00


In [3]:
stock_boxes = []
output = Output()
main_ui = VBox()

def add_stock_box():
    stock_box = Text(
        placeholder='Enter stock ticker (e.g., AAPL, TSLA)',
        layout=Layout(width='300px', padding='5px 10px'),
    )
    stock_boxes.append(stock_box)
    update_ui()

def update_ui():
    stock_layout = [
        HBox(
            [
                Label("Stock:", layout=Layout(width="150px")),
                stock_boxes[i],
            ],
            layout=Layout(margin="10px 0"),
        )
        for i in range(len(stock_boxes))
    ]

    main_ui.children = (
        [title_label]
        + stock_layout
        + [
            HBox(
                [
                    Label("Risk-Free Rate (%):", layout=Layout(width="150px")),
                    risk_free_rate_input,
                ],
                layout=Layout(margin="10px 0"),
            )
        ]
        + [
            HBox(
                [
                    Label("Risk Level:", layout=Layout(width="150px")),
                    risk_level_dropdown,
                ],
                layout=Layout(margin="10px 0"),
            )
        ]
        + [HBox([add_button, calc_button], layout=Layout(margin="20px 0"))]
        + [output]
    )

def calculate_efficient_frontier(event):
    with output:
        output.clear_output()

        stocks = [box.value.upper() for box in stock_boxes if box.value.strip()]
        if not stocks:
            print("Please enter valid stock tickers.")
            return

        try:
            risk_free_rate = float(risk_free_rate_input.value) / 100
        except ValueError:
            print("Please enter a valid risk-free rate.")
            return

        risk_level = risk_level_dropdown.value

        try:
            data = yf.download(stocks, start="2020-01-01", end="2024-12-20", progress=False)['Adj Close']
            log_returns = np.log(data / data.shift(1))
            mean_returns = log_returns.mean() * 252
            cov_matrix = log_returns.cov() * 252
        except Exception as e:
            print(f"Error fetching data: {e}")
            return

        num_portfolios = 250000
        results = np.zeros((3, num_portfolios))
        weights_record = []

        for i in range(num_portfolios):
            weights = np.random.random(len(stocks))
            weights /= np.sum(weights)
            weights_record.append(weights)

            portfolio_return = np.dot(weights, mean_returns)
            portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
            sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev

            results[0, i] = portfolio_return
            results[1, i] = portfolio_std_dev
            results[2, i] = sharpe_ratio

        if risk_level == "aggressive":
            risk_filter = results[1, :] > 0.5
        elif risk_level == "conservative":
            risk_filter = results[1, :] < risk_free_rate
        elif risk_level == "mixed":
            risk_filter = (results[1, :] >= risk_free_rate) & (results[1, :] <= 0.5)

        filtered_results = results[:, risk_filter]
        filtered_weights = [weights_record[i] for i, include in enumerate(risk_filter) if include]

        if not filtered_results.any():
            print("No portfolios match the selected risk level.")
            return

        max_sharpe_idx = np.argmax(filtered_results[2])
        max_sharpe_return = filtered_results[0, max_sharpe_idx]
        max_sharpe_std_dev = filtered_results[1, max_sharpe_idx]
        max_sharpe_weights = filtered_weights[max_sharpe_idx]

        print(f"\nMax Sharpe Ratio Portfolio:\n")
        for i, stock in enumerate(stocks):
            print(f"{stock}: {max_sharpe_weights[i]*100:.2f}%")
        print(f"\nExpected Annual Return: {max_sharpe_return*100:.2f}%")
        print(f"Annual Volatility (Risk): {max_sharpe_std_dev*100:.2f}%")
        print(f"Sharpe Ratio: {filtered_results[2, max_sharpe_idx]:.2f}\n")

        plt.figure(figsize=(10, 6))
        plt.scatter(filtered_results[1, :], filtered_results[0, :], c=filtered_results[2, :], cmap='viridis', marker='o', s=10, alpha=0.3)
        plt.colorbar(label='Sharpe Ratio')
        plt.scatter(max_sharpe_std_dev, max_sharpe_return, c='red', s=50, edgecolors='black', label='Max Sharpe Ratio Portfolio')
        plt.title('Efficient Frontier', fontsize=16, fontweight='bold')
        plt.xlabel('Volatility (Risk)', fontsize=12)
        plt.ylabel('Return', fontsize=12)
        plt.legend()
        plt.show()

title_label = Label(
    value="Efficient Frontier Calculator",
    layout=Layout(padding='20px', margin='20px 0'),
    style={'font_size': '100px'}
)

risk_free_rate_input = Text(
    placeholder='Enter Risk-Free Rate',
    layout=Layout(width='300px')
)

risk_level_dropdown = Dropdown(
    options=[
        ("Aggressive (>50% risk)", "aggressive"),
        ("Conservative (<Risk-Free Rate)", "conservative"),
        ("Mixed (between risk-free rate and 50%)", "mixed"),
    ],
    value="mixed",
    layout=Layout(width='300px')
)

add_button = Button(
    description='Add More Stocks',
    button_style='primary',
    layout=Layout(width='200px')
)
calc_button = Button(
    description='Calculate Efficient Frontier',
    button_style='success',
    layout=Layout(width='200px')
)

add_button.on_click(lambda b: add_stock_box())
calc_button.on_click(calculate_efficient_frontier)

for _ in range(3):
    add_stock_box()

update_ui()
display(main_ui)